# Resumen Parcial de NLP

Este resumen muestra una de las metodologías utilizadas actualmente para diseñar un algoritmo de NLP. En este caso, la tarea a resolver será la de clasificación de texto. 

Esta metodología consiste en lo siguiente: para una tarea determinada, primero se realiza una extracción del significado de los elementos que componen el texto y luego se diseña un modelo que toma como entrada dicha representación del significado. Este modelo es el que realiza la tarea propiamente dicha, pero no lo hace sobre los símbolos que componen el texto original, sino sobre una representación de los mismos en la que pueden verse algunas características semánticas entre ellas. 

Por ejemplo, para el [corpus de texto *Brown*](https://www.nltk.org/book/ch02.html), que contiene textos de 15 categorías diferentes (news, editorial, reviews, religion, hobbies, lore, belles lettres, government, learned, fiction, mystery, science fiction, adventure, romanc, humor), se buscará realizar un algoritmo que clasifique una frase en alguna de estas categorías. Este procedimiento se realiza de dos maneras distintas:

* Se entrena un clasificador de una capa (con salida Softmax) que tiene como entrada una frase, y como salida la probabilidad de cada una de las categorías mencionadas anteriormente.

* Se entrena el mismo clasificador que antes, con la diferencia que la entrada del modelo no son las palabras que componen la frase, sino la representación del significado de cada una de esas palabras (*word embedding*).

**TODO: EXPLICAR UN POCO MEJOR QUE EL SIGNIFICADO SE EXTRAE CON UN MODELO DE LENGUAJE.**

In [1]:
# Importamos PyTorch
import torch

# Importamos la libraría de utils de NLP
import sys
sys.path.insert(1, '../')
from nlp_utils import *

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package brown to
[nltk_data]     /home/lestien/Documents/BecaNLP/Programs/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [36]:
test_prop = .2

train_samples = []
test_samples = []

categories = brown.categories()
for c, category in enumerate(categories):
    sents = brown.sents(categories=category)
    categ_len = len(sents)
    test_size = int(test_prop * categ_len)
    train_size = categ_len - test_size
    rand_idx = torch.randperm(categ_len)
    for i in rand_idx[:train_size]:
        train_samples.append((sents[i], '<BEGINLABEL>' + category + '<ENDLABEL>'))
    for i in rand_idx[train_size:]:
        test_samples.append((sents[i], '<BEGINLABEL>' + category + '<ENDLABEL>'))
    
train_file = open('train.txt', 'w+')
test_file = open('test.txt', 'w+')

for sample in train_samples:
    text, label = sample
    for i in range(len(text)-1):
        train_file.write(text[i])
        train_file.write('<TS>')
    train_file.write(text[-1])
    train_file.write(label)

for sample in test_samples:
    text, label = sample
    for i in range(len(text)-1):
        test_file.write(text[i])
        test_file.write('<TS>')
    test_file.write(text[-1])
    test_file.write(label)
    
train_file.close()
test_file.close()

In [ ]:
train_file = open('train.txt', 'r')
test_file = open('test.txt', 'r')

train_s = []
test_s = []

train = train_file.read()
train = train.split('<ENDLABEL>')
for i in range(len(train)-1):
    sample = train[i].split('<BEGINLABEL>')
    train_s.append((sample[0].split('<TS>'), sample[1]))

test = test_file.read()
test = test.split('<ENDLABEL>')
for i in range(len(test)-1):
    sample = test[i].split('<BEGINLABEL>')
    test_s.append((sample[0].split('<TS>'), sample[1]))

train_file.close()
test_file.close()

In [41]:
class BrownClassificationDataset(torch.utils.data.Dataset):
    
    def __init__(self, root, preprocessing, train=True):
        
        self.root = root
        
        if train:
            file = open(root + 'train.txt', 'r')
            text = file.read()
            file.close()
        else:
            file = open(root + 'test.txt', 'r')
            text = file.read()
            file.close()

        samples = []
        text = text.split('<ENDLABEL>')
        for i in range(len(text)-1):
            sample = text[i].split('<BEGINLABEL>')
            samples.append((sample[0].split('<TS>'), sample[1]))
               
        if preprocessing:
            samples_preprocessed = preprocessing(samples)
        else:
            samples_preprocessed = samples
            
        self.samples = samples_preprocessed
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]
    
    
train_dataset = BrownClassificationDataset('./', None, train=True)
val_dataset = BrownClassificationDataset('./', None, train=True)
test_dataset = BrownClassificationDataset('./', None, train=False)

In [42]:
print(len(train_dataset))
print(train_dataset[:5])

45879
[(['Why', ',', "that's", 'his', 'main', 'reason', 'for', 'making', 'the', 'dive', "''", '.'], 'adventure'), (['The', 'enemy', 'came', 'looming', 'around', 'a', 'bend', 'in', 'the', 'trail', 'and', 'Matsuo', 'took', 'a', 'hasty', 'shot', ',', 'then', 'fled', 'without', 'knowing', 'the', 'result', ',', 'ran', 'until', 'breath', 'was', 'a', 'pain', 'in', 'his', 'chest', 'and', 'his', 'legs', 'were', 'rubbery', '.'], 'adventure'), (['He', 'must', 'have', 'saturated', 'himself', 'in', 'the', 'drink', ',', 'for', 'the', 'bullet', 'not', 'to', 'shock', 'him', 'out', 'of', 'his', 'drunken', 'haze', '.'], 'adventure'), (["Montero's", 'shot', 'had', 'caught', 'him', 'high', 'in', 'the', 'chest', ';', ';'], 'adventure'), (['``', "Let's", 'get', 'one', 'thing', 'straight', ',', 'you', 'and', 'me', '.'], 'adventure')]


## Primera forma (sin extracción del significado)

In [ ]:
val_prop = .05
test_prop = .15

train, train_labels = [], []
val, val_labels = [], []
test, test_labels = [], []

for c, category in enumerate(brown.categories()):
    sents = brown.sents(categories=category)
    categ_len = len(sents)
    val_size = int(val_prop * categ_len)
    test_size = int(test_prop * categ_len)
    train_size = categ_len - val_size - test_size
    rand_idx = torch.randperm(categ_len)
    for i in rand_idx[:train_size]:
        train.append(sents[i])
        train_labels.append(c)
    for i in rand_idx[train_size:(train_size+val_size)]:
        val.append(sents[i])
        val_labels.append(c)
    for i in rand_idx[-test_size:]:
        test.append(sents[i])
        test_labels.append(c)

train_samples, train_rand_idx = [], torch.randperm(len(train))
val_samples, val_rand_idx = [], torch.randperm(len(val))
test_samples, test_rand_idx = [], torch.randperm(len(test))

for i in train_rand_idx:
    train_samples.append((train[i], train_labels[i]))
for i in val_rand_idx:
    val_samples.append((val[i], val_labels[i]))
for i in test_rand_idx:
    test_samples.append((test[i], test_labels[i]))

In [ ]:

        





test_prop = .2

train_samples = []
test_samples = []

for c, category in enumerate(brown.categories()):
    sents = brown.sents(categories=category)
    categ_len = len(sents)
    test_size = int(test_prop * categ_len)
    train_size = categ_len - test_size
    rand_idx = torch.randperm(categ_len)
    for i in rand_idx[:train_size]:
        train_samples.append((sents[i], c))
    for i in rand_idx[train_size:]:
        test_samples.append((sents[i],c))

In [4]:
# TODO: INVESTIGAR Y HACER UN BUEN PREPROCESAMIENTO!!
class PreprocessBrown(object):
    def __call__(self,corpus_unpreproceced):
        corpus = []
        for sentence in corpus_unpreproceced:
            text = ' '.join(sentence)
            text = text.lower()
            text.replace('\n', ' ')
            text = re.sub('[^a-z ]+', '', text)
            corpus.append([w for w in text.split() if w != ''])
        return corpus

nltk.corpus.util.LazyCorpusLoader

## Segunda forma (con extracción del significado)